# 作业 

In [1]:
import numpy as np 
from scipy.special import softmax 
from scipy.stats import bernoulli
import time 
from scipy.optimize import minimize
from ddm import pdf # ddm probility density function

from IPython.display import clear_output
%matplotlib inline
%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt 
import seaborn as sns 

import sys 
sys.path.append("..") 
from utils.env import frozen_lake
from utils.viz import viz 
viz.get_style()

本次作业主要涉及两种参数估计方法(MLE+MCMC)和强化学习(dynamic programming+Sarsa)

## Part 1 参数估计之MLE
* 采用`exampledata.txt`的数据

In [ ]:
# new data
data = np.loadtxt('example.txt')
print(data.shape)

###

**Q1.1: 请写出ddm的负对数似然函数`negloglikeli`(negative log-likelihood function)**


In [ ]:
# define a negative log-likelihood objective functions
def negloglikeli(params):
    '''
    <params>:(4,) array, drift coefficient, decision boundary, initial bias, non-decision time
    '''
    # specify your loaded params
    ##------------------------------##
    ##           your answer        ##
    ##------------------------------##
    # do initialization    
    ##------------------------------##
    ##           your answer        ##
    ##------------------------------##
    # loop trial
    ##------------------------------##
    ##           your answer        ##
    ##------------------------------##
    pp=0.999*pp + np.finfo(np.float32).eps # to avoid p=0
    # take log, sum，add negative
    return ##      your answer      ##


###

**Q1.2: 利用最大似然估计(maximum likelihood estimation)的方法求解ddm模型参数k,b.a,ndt**
* 参数k,b.a,ndt的bounds分别为((0, 20), (0, 5), (0, 1), (0, 1))
* 请使用example.txt。其中1-3列分别是：coherence(运动序列。为了简化，只有一种coherence)，response time(总反应时长)，correct(作答是否正确)
* example.txt的形状为(3000,3)


In [ ]:
np.random.seed(1234)
## now start your MLE

# print your result
print('\nfitted drift coefficient is ', res.x[0])
print('fitted decision boundary is ', res.x[1])
print('fitted initial bias is ', res.x[2])
print('fitted nondecision time is ', res.x[3])

正确答案如下：

fitted drift coefficient is  1.0311682099419661

fitted decision boundary is  2.3827290342577285

fitted initial bias is  0.4972358161521576

fitted nondecision time is  0.20728465380894037

## Part 2 参数估计之MCMC

###
**Q2: 利用Metropolitan-Hasting的方法估计DDM模型中的参数：drift rate .不能借助工具包，必须手写M-H**
* 请使用example.txt。
* 将DDM中其余参数固定为： 
    - B(boundary) = 2
    - a(initial bias) = 0.5
    - ndt(non-decision time) = 0.1
* (代码中已经固定randomseed，每一次结果都将一致。请勿修改)
* hint：
    - 提议分布建议使用正态分布。正态分布可以使用scipy.stats.norm()
    - 从提议分布中每一次采样，起始值都设置为每一次循环初始的initial值
    - 从提议分布中采样的结果通过exp()转换（因为drift rate始终大于零）
    - $min[\frac{f(x)}{f(y)}, 1] = min[log(f(x))-log(f(y)), 0]$

In [ ]:
from scipy.stats import norm, uniform

def metropolis_hastings(data, y_k, n_sample=30000, n_burnin=10000, random_seed=1234):
    np.random.seed(random_seed)
    ##---------------------------------------##
    ##               your answer             ##
    ##---------------------------------------##
    for i in range(n_sample + n_burnin):
        ##---------------------------------------##
        ##               your answer             ##
        ##---------------------------------------##  
    return np.array(samples)


# 开始M-H采样
samples = metropolis_hastings(data)

In [ ]:
plt.figure(figsize = (6,4))
plt.hist(samples, bins=15, density=True, label='samples')
plt.xlabel('drift rate')
plt.ylabel('Density')
plt.title(' Estimation of Samples')
plt.show()

正确答案如下图

<img src='MH.png' width='400' height='300'>

## Part 3 强化学习之dynamic programming

### 
我们接下来的游戏需要借助一个冰湖任务完成。这个游戏的规则很简单：掉进蓝色冰窟窿内即失败(游戏结束)；顺利到达goal(G)即成功。
在这个冰湖环境中，agent的动作为：
* 0: 上
* 1: 下
* 2: 左
* 3: 右
##### 先来看看我们已经拥有的冰湖任务的环境。

In [ ]:
seed = 1234
env = frozen_lake(seed=seed)
env.reset()
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
env.render(ax)

###
**Q3：完成value iteration。**
完成value iteration的步骤(伪代码)：
1. Initialize V(s) arbitrarily for all s ∈ States(except termination)
2. loop 
    1. $ Δ ← 0$
    2. loop for each state s ∈ States
        1. $ v ← V(s)$
        2. $ V(s) ← max_a Σ_s' p(s'|s, a) * [R(s') + \gamma * V(s')]$
        3. $Δ ← max(Δ, |v - V(s)|)$
3. loop until $Δ < \theta$
4. $ pi(s) = argmax_a Σ_{s'} p(s'|s, a) * [R(s') + \gamma * V(s')]$
5. return $V,pi$
    
hint：
* 可以使用env.r()获取下一个状态的reward
* 可以使用env.p_s_next()来获取状态转移概率
* 可以借助np.eye()来找到能返回最大值的action



In [ ]:
def value_iter(env, seed=1234, theta=1e-4, gamma=.99):
    
    rng = np.random.RandomState(seed)
    # initialize V(s), arbitrarily except V(terminal)=0
    V = np.zeros(env.nS)
    if s in env.s_termination:
        V[s] = 0
    # init policy 
    pi = softmax(rng.randn(env.nS,env.nA)*5, axis=1)    
    # loop until converge
    while True:
        delta = 0
        for s in env.S:
            v_old = V[s].copy()
            v_new = 0
            for a in env.A:
                p = env.p_s_next(s, a)
                for s_next in env.S:
                    # calculate v_new
                    ###-------------------------------###
                    ###         your answer           ###
                    ###-------------------------------###
            # calculate V
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------###
            # get new policy 
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------###
            # calculate delta
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------###
        
        # visualize 
        if show_update:
            _, axs = plt.subplots(1, 2, figsize=(8, 4))
            clear_output(True)
            ax = axs[0]
            env.show_v(ax, V)
            ax = axs[1]
            env.show_pi(ax, pi)
            time.sleep(.1)
            plt.show()
        # check covergence 
        if delta < theta:
            break 
    for s in env.s_termination:
        V[s] = 0
    return V, pi 

## Part 4 强化学习中之 sarsa (TD-learning)
上节课我们讲的内容是Q-learning。SARSA和Q-learning都属于强化学习中的TD-learning范畴，因为它们都使用了时序差分学习(TD-learning)的思想来更新值函数。虽然它们的目标相同，但它们在更新策略上有一些不同之处。

1. **目标**：
   - SARSA的目标是学习最优的动作值函数Q，并根据该值函数选择动作。它基于当前状态-动作对的值更新。
   - Q-learning的目标是学习最优的状态值函数Q，并根据该值函数选择动作。它基于当前状态下所有可能动作的最大值更新。

2. **更新策略**：
   - 在SARSA中，Q值的更新考虑了当前状态下采取的动作以及在下一个状态下采取的动作。具体地说，更新公式为：
            $Q_{(s,a)} = Q_{(s,a)} + \alpha * [r + \gamma*Q_{(s',a')}-Q_{(s,a)}]$
   - 其中，\(s\)是当前状态，\(a\)是当前动作，\(r\)是获得的奖励，\(s'\)是下一个状态，\(a'\)是下一个状态下采取的动作，\(α\)是   学习率，\(γ\)是折扣因子。
   - 在Q-learning中，Q值的更新只考虑了当前状态下所有可能的动作中的最大值。



正确答案如下图

<img src='value_and_policy.png' width='800' height='400'>

In [ ]:
def e_greedy(q, rng, env, eps):
    a_max = np.argwhere(q==np.max(q)).flatten()
    policy = np.sum([np.eye(env.nA)[i] for i in a_max], axis=0) / len(a_max)
    if rng.rand() < 1-eps:
        a = rng.choice(env.nA, p=policy)
    else:
        a = rng.choice(env.nA)
    return a 

### 
**Q4：补充sarsa 代码**

我们来看一下实现Sarsa的步骤：
1. for episode = 1 to episodes :
    1. Initialize state s
    2. Choose action a using ε-greedy policy based on Q(s, a)
    3. loop
        1. Take action a, observe reward r and next state s'
        2. Choose action a' using ε-greedy policy based on Q(s', a')
        3. Q(s, a) ← Q(s, a) + α * [r + γ * Q(s', a') - Q(s, a)] 
        4. s ← s'
        5. a ← a'
    4. loop until s is terminal

2. episode loop 结束，return Q

hint
* episode是指训练的总次数
*  这一步：Q(s, a) ← Q(s, a) + α * [r + γ * Q(s', a') - Q(s, a)] ，需要考虑下一个状态是否为done(终点)。
    - 如果是：Q(s, a) ← Q(s, a) + α * [r + γ * 0 - Q(s, a)]
    - else：Q(s, a) ← Q(s, a) + α * [r + γ * Q(s', a') - Q(s, a)]

In [ ]:
def Sarsa(env, alpha=.1, eps=.1, gamma=.99, max_epi=2000, seed=1234, theta=1e-4):
    # rng
    rng = np.random.RandomState(seed)
    Q = np.zeros([env.nS, env.nA])
    for epi in range(max_epi):
        s, r, done = env.reset()
        q_old = Q.copy()
        G = 0
        while True:
            # sample At, observe Rt, St+1
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------### 
            # calc s_next
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------### 
            # given s_next, calc a_next
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------###   
            # update Q,s,a
            ###-------------------------------###
            ###         your answer           ###
            ###-------------------------------###       
            G += r                          
            if done:
                break     
        if (np.abs(q_old - Q)<theta).all():
            break
    pi = np.eye(env.nA)[np.argmax(Q, axis=1)]
    return Q, pi

In [ ]:
## 可视化
Q_sarsa, pi_sarsa = Sarsa(env)
V3 = Q_sarsa.max(1)
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
ax = axs[0]
env.show_v(ax, V3)
ax = axs[1]
env.show_pi(ax, pi_sarsa)

正确答案如下图

<img src="sarsa.png" width='800' height='400'>
